In [1]:
model_name = "roberta-base"

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()

wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
sweep_config = {
   'method': 'grid',
   'parameters': {
       'epochs': {
           'value': 3
       },
       'learning_rate': {
           'values': [5e-5, 4e-5, 3e-5, 2e-5]
       },
       'batch_size': {
           'value': 16
       },
       'weight_decay': {
           'values': [0.1, 0.01, 0.0]
       },
       'warmup_ratio': {
           'value': 0.1
       }
   }
}
project = 'RoBERTa-base-goemotions-seed-2'
sweep_id = wandb.sweep(sweep_config, project=project)

Create sweep with ID: j20ua06q
Sweep URL: https://wandb.ai/martinsit288/RoBERTa-base-goemotions-seed-2/sweeps/j20ua06q


In [5]:
import transformers
import datasets

print(f"Running on transformers v{transformers.__version__} and datasets v{datasets.__version__}")

Running on transformers v4.20.1 and datasets v2.1.0


In [6]:
!pip3 install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.2 MB/s eta 0:00:00


In [7]:
import torch
from pathlib import Path
import evaluate
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)
import numpy as np
import random
import os

## Seed everything

In [8]:
SEED = 2

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

## Load dataset

In [10]:
ds = load_dataset("go_emotions", "simplified")
ds

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [11]:
ds["train"][0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [27],
 'id': 'eebbqej'}

In [12]:
def one_hot_encode(example):
    l = example["labels"]
    one_hot_list = [0] * (28)
    for i in l:
        one_hot_list[i] = 1
    example["labels"] = one_hot_list
    return example

In [13]:
ds = ds.map(one_hot_encode)

  0%|          | 0/43410 [00:00<?, ?ex/s]

  0%|          | 0/5426 [00:00<?, ?ex/s]

  0%|          | 0/5427 [00:00<?, ?ex/s]

In [14]:
ds["train"][0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 'id': 'eebbqej'}

In [15]:
id2label = {0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}
label2id = {'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'caring': 5, 'confusion': 6, 'curiosity': 7, 'desire': 8, 'disappointment': 9, 'disapproval': 10, 'disgust': 11, 'embarrassment': 12, 'excitement': 13, 'fear': 14, 'gratitude': 15, 'grief': 16, 'joy': 17, 'love': 18, 'nervousness': 19, 'optimism': 20, 'pride': 21, 'realization': 22, 'relief': 23, 'remorse': 24, 'sadness': 25, 'surprise': 26, 'neutral': 27}

## Tokenize and encode

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [17]:
def tokenize_func(examples):
  return tokenizer(examples["text"], truncation=True, max_length=50)

In [18]:
ds_enc = ds.map(tokenize_func, batched=True, remove_columns=["text", "id"])
ds_enc = ds_enc.rename_column("labels", "label")
ds_enc

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5427
    })
})

In [19]:
def to_float_labels(example):
    float_labels = example["label"].to(torch.float)
    example["float_label"] = float_labels
    return example

In [20]:
ds_enc.set_format("torch")
ds_enc = ds_enc.map(to_float_labels).remove_columns("label").rename_column("float_label", "label")

  0%|          | 0/43410 [00:00<?, ?ex/s]

  0%|          | 0/5426 [00:00<?, ?ex/s]

  0%|          | 0/5427 [00:00<?, ?ex/s]

In [21]:
ds_enc["train"][0]

{'input_ids': tensor([   0, 2387, 5548,  689,   16,  932,   38,  399,   75,   33,    7, 7142,
         2185,    4,    2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'label': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}

## Load model

In [22]:
def model_init():
    num_labels = 28
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True, problem_type='multi_label_classification').to(device)
    return model

## Load trainer

In [23]:
acc = evaluate.load("accuracy", "multilabel")
f1 = evaluate.load("f1", "multilabel")
precision = evaluate.load("precision", "multilabel")
recall = evaluate.load("recall", "multilabel")

In [24]:
from sklearn.metrics import hamming_loss
def hamming_score(predictions, references):
    return 1 - hamming_loss(references, predictions)

In [25]:
def compute_metrics(eval_pred, threshold=0.3):
    prediction_scores, labels = eval_pred
    
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.tensor(prediction_scores))
    
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= threshold)] = 1
    
    return {
        'hamming': hamming_score(predictions=predictions, references=labels),
        'exact_match': acc.compute(predictions=predictions, references=labels)['accuracy'],
        'macro_f1': f1.compute(predictions=predictions, references=labels, average="macro")['f1'],
        'macro_precision': precision.compute(predictions=predictions, references=labels, average="macro")['precision'],
        'macro_recall': recall.compute(predictions=predictions, references=labels, average="macro")['recall'],
        'micro_f1': f1.compute(predictions=predictions, references=labels, average="micro")['f1'],
        'micro_precision': precision.compute(predictions=predictions, references=labels, average="micro")['precision'],
        'micro_recall': recall.compute(predictions=predictions, references=labels, average="micro")['recall'],
        'weighted_f1': f1.compute(predictions=predictions, references=labels, average="weighted")['f1'],
        'weighted_precision': precision.compute(predictions=predictions, references=labels, average="weighted")['precision'],
        'weighted_recall': recall.compute(predictions=predictions, references=labels, average="weighted")['recall']
    }

In [26]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        training_args = TrainingArguments(
            output_dir="results",
            report_to="wandb",
            num_train_epochs=config.epochs,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            warmup_ratio=config.warmup_ratio,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=16,
            load_best_model_at_end=True,
            logging_strategy="steps",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            seed=SEED
        )
        
        trainer = Trainer(
            model_init=model_init, 
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=ds_enc["train"],
            eval_dataset=ds_enc["validation"],
            tokenizer=tokenizer
        )
        
        trainer.train()
        
        trainer.evaluate(eval_dataset=ds_enc["test"], metric_key_prefix="test")

In [27]:
wandb.agent(sweep_id, train, count=12, project=project)

wandb: Agent Starting Run: w236as4p with config:
wandb: 	batch_size: 16
wandb: 	epochs: 3
wandb: 	learning_rate: 5e-05
wandb: 	warmup_ratio: 0.1
wandb: 	weight_decay: 0.1
wandb: Currently logged in as: martinsit288. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarrassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "joy",
    "18": "love",
    "

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
creating metadata file for /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.poole

...

PyTorch: setting up devices
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarrassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "j

Epoch,Training Loss,Validation Loss,Hamming,Exact Match,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Weighted F1,Weighted Precision,Weighted Recall
1,0.098100,0.092345,0.965918,0.437154,0.376869,0.469098,0.373794,0.587739,0.597249,0.578527,0.544653,0.570067,0.578527
2,0.085100,0.085032,0.966438,0.457427,0.459314,0.496478,0.463883,0.605798,0.597712,0.614107,0.588603,0.589064,0.614107
3,0.074900,0.083345,0.966530,0.457796,0.473533,0.488148,0.478172,0.610315,0.597091,0.624138,0.597690,0.584915,0.624138


***** Running Evaluation *****
  Num examples = 5426
  Batch size = 16
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to results/checkpoint-2714
Configuration saved in results/checkpoint-2714/config.json
Model weights saved in results/checkpoint-2714/pytorch_model.bin
tokenizer config file saved in results/checkpoint-2714/tokenizer_config.json
Special tokens file saved in results/checkpoint-2714/special_tokens_map.json
****

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


eval/exact_match,▁██
eval/hamming,▁▇█
eval/loss,█▂▁
eval/macro_f1,▁▇█
eval/macro_precision,▁█▆
eval/macro_recall,▁▇█
eval/micro_f1,▁▇█
eval/micro_precision,▃█▁
eval/micro_recall,▁▆█
eval/runtime,█▇▁
eval/samples_per_second,▁▂█
